Import Libraries

In [2]:
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import regularizers

!pip install -q git+https://github.com/tensorflow/docs
import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots

Load and prepare dataset

In [3]:
gz = tf.keras.utils.get_file('HIGGS.csv.gz', 'http://mlphysics.ics.uci.edu/data/higgs/HIGGS.csv.gz')

2816417792/2816407858 [==============================] - 66s 0us/step


In [4]:
FEATURES = 28
ds = tf.data.experimental.CsvDataset(gz,[float(),]*(FEATURES+1), compression_type="GZIP")

def pack_row(*row):
  label = row[0]
  features = tf.stack(row[1:],1)
  return features, label

packed_ds = ds.batch(10000).map(pack_row).unbatch()

In [5]:
N_VALIDATION = int(1e3)
N_TRAIN = int(1e4)
BUFFER_SIZE = int(1e4)
BATCH_SIZE = 500
STEPS_PER_EPOCH = N_TRAIN//BATCH_SIZE

In [6]:
validate_ds = packed_ds.take(N_VALIDATION).cache()
train_ds = packed_ds.skip(N_VALIDATION).take(N_TRAIN).cache()
validate_ds = validate_ds.batch(BATCH_SIZE)
train_ds = train_ds.shuffle(BUFFER_SIZE).repeat().batch(BATCH_SIZE)

Simple model

In [16]:
model = tf.keras.Sequential([
    layers.Dense(16, activation='elu', input_shape=(FEATURES,)),
    layers.Dropout(0.5),
    layers.Dense(256, activation='elu'),
    layers.Dropout(0.4),
    layers.Dense(1024, activation='elu'),
    layers.Dropout(0.4),
    layers.Dense(128, activation='elu'),
    layers.Dropout(0.2),
    layers.Dense(1)
])

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 16)                464       
                                                                 
 dropout_12 (Dropout)        (None, 16)                0         
                                                                 
 dense_21 (Dense)            (None, 256)               4352      
                                                                 
 dropout_13 (Dropout)        (None, 256)               0         
                                                                 
 dense_22 (Dense)            (None, 1024)              263168    
                                                                 
 dropout_14 (Dropout)        (None, 1024)              0         
                                                                 
 dense_23 (Dense)            (None, 128)              

In [8]:
def get_callbacks():
  return [
    tfdocs.modeling.EpochDots(),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=200),
  ]

In [9]:
def compile_and_fit(model, max_epochs=10000):
  model.compile(optimizer="adam",
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=[
                  tf.keras.losses.BinaryCrossentropy(
                      from_logits=True, name='binary_crossentropy'),
                  'accuracy'])

  model.summary()

  history = model.fit(
    train_ds,
    steps_per_epoch = STEPS_PER_EPOCH,
    epochs=max_epochs,
    validation_data=validate_ds,
    callbacks=get_callbacks(),
    verbose=0)
  return history

In [ ]:
compile_and_fit(model)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 16)                464       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 17        
Total params: 481
Trainable params: 481
Non-trainable params: 0
_________________________________________________________________

Epoch: 0, accuracy:0.4830,  binary_crossentropy:0.7695,  loss:0.7695,  val_accuracy:0.4710,  val_binary_crossentropy:0.7413,  val_loss:0.7413,  
....................................................................................................
Epoch: 100, accuracy:0.6005,  binary_crossentropy:0.6250,  loss:0.6250,  val_accuracy:0.5600,  val_binary_crossentropy:0.6285,  val_loss:0.6285,  
....................................................................................................
Epoch: 200, accuracy:0.

In [10]:
# Solve Under Fitting
compile_and_fit(model)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                464       
                                                                 
 dense_1 (Dense)             (None, 256)               4352      
                                                                 
 dense_2 (Dense)             (None, 1024)              263168    
                                                                 
 dense_3 (Dense)             (None, 128)               131200    
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 399,313
Trainable params: 399,313
Non-trainable params: 0
_________________________________________________________________

Epoch: 0, accuracy:0.5058,  binary_crossentropy:0

In [12]:
# Solve Over Fitting -> Dropout
compile_and_fit(model)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 16)                464       
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_6 (Dense)             (None, 256)               4352      
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_7 (Dense)             (None, 1024)              263168    
                                                                 
 dropout_2 (Dropout)         (None, 1024)              0         
                                                                 
 dense_8 (Dense)             (None, 128)              

In [15]:
# Solve Over Fitting -> Add L2
compile_and_fit(model)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 16)                464       
                                                                 
 dropout_8 (Dropout)         (None, 16)                0         
                                                                 
 dense_16 (Dense)            (None, 256)               4352      
                                                                 
 dropout_9 (Dropout)         (None, 256)               0         
                                                                 
 dense_17 (Dense)            (None, 1024)              263168    
                                                                 
 dropout_10 (Dropout)        (None, 1024)              0         
                                                                 
 dense_18 (Dense)            (None, 128)              

In [17]:
# Solve L2 Adverse Effect 
compile_and_fit(model)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 16)                464       
                                                                 
 dropout_12 (Dropout)        (None, 16)                0         
                                                                 
 dense_21 (Dense)            (None, 256)               4352      
                                                                 
 dropout_13 (Dropout)        (None, 256)               0         
                                                                 
 dense_22 (Dense)            (None, 1024)              263168    
                                                                 
 dropout_14 (Dropout)        (None, 1024)              0         
                                                                 
 dense_23 (Dense)            (None, 128)              